In [3]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler

In [7]:
df = pd.read_csv("../../train.csv")
# df.columns

In [8]:
# # H-N
# d1 = df[df.direction==1][["LONGITUDE", "LATITUDE","FC", "trip_id"]] # h-n
# d2 = df[df.direction==0][["LONGITUDE", "LATITUDE","FC", "trip_id"]] # n-h

# # get the avg location of the top 1 percent trip
# def get_top1_location(d1):
#     total = d1.trip_id.nunique() // 100
#     top_1_trips = list(d1.groupby("trip_id").FC.sum().sort_values()[:total].index)
#     i = top_1_trips.pop(0)
#     hn_top = d1[d1.trip_id==i].reset_index(drop=True)
#     if (len(hn_top) < 200):
#         for j in range(200-len(hn_top)):
#             hn_top = hn_top.append(hn_top.tail(1)).reset_index(drop=True)
#     for i in top_1_trips:
#         tmp_df2 = df[df.trip_id==i].reset_index(drop=True)
#         if (len(tmp_df2) < 200):
#             for j in range(200-len(tmp_df2)):
#                 tmp_df2 = tmp_df2.append(tmp_df2.tail(1)).reset_index(drop=True)
#         hn_top["LONGITUDE"] = hn_top.LONGITUDE + tmp_df2.LONGITUDE
#         hn_top["LATITUDE"] = hn_top.LATITUDE + tmp_df2.LATITUDE
#     hn_top["LONGITUDE"] = hn_top["LONGITUDE"]/(len(top_1_trips)+1)
#     hn_top["LATITUDE"] = hn_top["LATITUDE"]/(len(top_1_trips)+1)
#     return hn_top[["LONGITUDE", "LATITUDE"]]

In [9]:
# hn_top = get_top1_location(d1)
# nh_top = get_top1_location(d2)

# hn_top.to_csv("H2N_top1.csv", index=False)
# nh_top.to_csv("N2H_top1.csv", index=False)

In [10]:
hn_top = pd.read_csv("../data/H2N_top1.csv")
nh_top = pd.read_csv("../data/N2H_top1.csv")

In [11]:
# hn_top_np = np.zeros((len(hn_top), 18))
# hn_top_np[:,13] = hn_top["LATITUDE"]
# hn_top_np[:,14] = hn_top["LONGITUDE"]
# hn_top_np = minmax_scaler.transform(hn_top_np)
# hn_top["LATITUDE"] = hn_top_np[:,13]
# hn_top["LONGITUDE"] = hn_top_np[:,14]

# nh_top_np = np.zeros((len(nh_top), 18))
# nh_top_np[:,13] = nh_top["LATITUDE"]
# nh_top_np[:,14] = nh_top["LONGITUDE"]
# nh_top_np = minmax_scaler.transform(nh_top_np)
# nh_top["LATITUDE"] = nh_top_np[:,13]
# nh_top["LONGITUDE"] = nh_top_np[:,14]
# nh_top

In [12]:
def get_observations(df, feature_cols = ["Time2", "turn", "acceleration",
       "distance", 'current', 'rain', 'snowfall', 'wind_force', 'wind_direc', "resist_ratio",
       'change_x_factor', 'change_y_factor', 
       "is_weekday", 'direction', "season", "hour", 
       "FC", "SOG", "LATITUDE", 'LONGITUDE',
       ], 
       action_cols = ["SPEED", "HEADING", "MODE"]):
    # df = df.fillna(0)
    
    features = df.copy()

    rewards_col = ["trip_id", "LONGITUDE", "LATITUDE", "direction", "goal_long", "goal_lat"]
    rewards_df = features[rewards_col]
    # apply minmax scaler for fuel consumption to get reward 2
    rewards_df["reward2"] = - features[["FC"]]

    dataset_list = []
    for i in list(df.trip_id.unique()):
        data_dict = {}
        # observations
        observation = features[features.trip_id==i].drop("trip_id", axis=1)[feature_cols]
        observation = observation.to_numpy().astype(float)
        data_dict["observations"] = observation

        # next_observations
        observation = np.delete(observation, 0, 0)
        last = observation[-1]
        observation = np.vstack([observation, last])
        data_dict["next_observations"] = observation

        # actions 
        # print(i)
        actions = features[features.trip_id==i].drop("trip_id", axis=1)[action_cols]
        # actions = actions.drop("trip_id", axis=1)
        data_dict["actions"] = actions.to_numpy().astype(float)

        # rewards
        rewards = rewards_df[rewards_df.trip_id==i].reset_index()
        rewards
        # reward1 distance to top 1
        trip_len = rewards.shape[0]
        if rewards.loc[0,"direction"]==1:
            top1 = hn_top.iloc[:trip_len]
        else:
            top1 = nh_top.iloc[:trip_len]
        rewards["reward1"] = - ((rewards["LONGITUDE"]-top1["LONGITUDE"])**2 + \
                             (rewards["LATITUDE"]-top1["LATITUDE"])**2 )**0.5
        rewards["reward1"].apply(lambda x: 0 if x > -0.05 else x)
        rewards["reward1"] = rewards["reward1"]
        # reward2 fc consumption an ddone reward
        rewards.loc[len(rewards)-1,"reward2"] = rewards.loc[len(rewards)-1,"reward2"]+3
        # rewards4 time out reward
        rewards["Time"] = rewards.index
        rewards.reset_index(inplace=True)
        rewards["reward3"] = rewards["Time"].apply(lambda x: -0.1*((x-90)//10) if x > 100 else 0)

        data_dict["rewards"] = (rewards[["reward1","reward2", "reward3"]]).to_numpy()

        # termination
        termination = np.zeros([observation.shape[0],1])
        termination[-1,0] = 1
        data_dict["termination"] = termination

        dataset_list.append(data_dict)
    return dataset_list

In [13]:
rl_data = get_observations(df)
with open('../data/rl_data.pkl', 'wb') as handle:
    pickle.dump(rl_data,handle)

/Users/yim/.local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
